In [7]:
import IO
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
men_patients = IO.load_excel("./data/Meniskus.xlsx").dropna()
men_amount = men_patients.shape[0]
kreuz_patients = IO.load_excel("./data/Kreuzband.xlsx").dropna()
df = pd.concat([men_patients, kreuz_patients], ignore_index=True)
df.head()

,Patientid_x,Text_Bef,Text_ANA,Geschlecht,Diagnose
0,89078.0,"li Knie: diskr. Erguss, kein DS med. GS, kein ...","Schmerzen werden besser, geht heute das dritte...",männlich,Meniskus
1,76793.0,"Re. Knie: deutlicher Erguss, 0-5-110, reizlose...",Am 02.03. beim Trampolinspringen Distorsionstr...,männlich,Meniskus
2,409850.0,"Linkes Kniegelenk: Extension/Flexion 0/3/100°,...","wv mit aktuellem MRT. Beugung besser, Streckun...",männlich,Meniskus
3,76793.0,"linke Schulter: GhAbd. 90°, Aro 60°, Iro LWK 1...",Patient berichtet er habe zwischenzeitlich ern...,männlich,Meniskus
4,91686.0,"re.Knie: ROM 0-0-140, Lachman neg. fest, DS me...","Belastungserprobung mit Rennrad und Laufen, bi...",männlich,Meniskus


In [3]:
# List of strings that should not be split
do_not_split = [
    "li.",
    "bds.",
    "med.",
    "neg.",
    "lat.",
    'pos.',
    "re.",
    "mm, links",
    "mm, rechts",
    "mm, li",
    "mm, re",
    "o.B",
    "mm, verletzt",
    "mm, Seitendifferenz",
    "diskr.",
    "flüssig und mit"
]

In [4]:
def custom_split(text, exceptions):
    for exception in exceptions:
        text = re.sub(re.escape(exception), re.sub(r'[^\w\s]', '', exception), text, flags=re.IGNORECASE)
    parts = re.split(r'[.,]|\sund\s', text)
    parts = [part.strip() for part in parts if part.strip()]
    return parts

In [5]:
# Preprocess the text
pattern_pre = r'\d{1,3}/\d{1,3}/\d{1,3}|\d{1,3}-\d{1,3}-\d{1,3}'

def adaptText(match):
    # Extract the matched text
    matched_text = match.group(0)
    
    # Split the matched text by either '/' or '-'
    if '/' in matched_text:
        parts = matched_text.split('/')
    elif '-' in matched_text:
        parts = matched_text.split('-')
    
    # Convert parts to integers
    parts = list(map(int, parts))
    
    # Evaluate each part in the context of the "Neutral-Null-Methode"
    hyperextension = parts[0]
    extension_deficit = parts[1]
    flexion = parts[2]
    
    # Create descriptions based on the measurements
    if hyperextension > 0:
        hyperextension_desc = "überstreckung des knies ist möglich,"
    else:
        hyperextension_desc = "keine überstreckung des knies,"
    
    if extension_deficit > 0:
        extension_deficit_desc = "streckdefizit des knies,"
    else:
        extension_deficit_desc = "kein streckdefizit des knies,"
    
    if flexion >= 120:
        flexion_desc = "gute beugung des knies,"
    elif flexion >= 90:
        flexion_desc = "angemessene beugung des knies,"
    else:
        flexion_desc = "eingeschränkte beugung des knies,"
    
    # Combine the descriptions into a final string
    result = f",{hyperextension_desc} {extension_deficit_desc} {flexion_desc}"
    return result

def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub('ds', 'druckschmerz', text, flags=re.IGNORECASE)
    text = re.sub('medial', 'innere', text, flags=re.IGNORECASE)
    text = re.sub('gelenkspalt', 'seite des kniegelenks', text, flags=re.IGNORECASE)



    text = re.sub('lateral', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('lat', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('gs', 'seite des kniegelenks', text, flags=re.IGNORECASE)
    text = re.sub('li ', 'links ', text, flags=re.IGNORECASE)
    text = re.sub('re ', 'rechts ', text, flags=re.IGNORECASE)
    text = re.sub('gb ', 'gangbild ', text, flags=re.IGNORECASE)
    text = re.sub('patellafacette', 'kniescheibe', text, flags=re.IGNORECASE)
    text = re.sub('patella', 'kniescheibe', text, flags=re.IGNORECASE)


    text = re.sub(pattern_pre, adaptText, text, flags=re.IGNORECASE)

    text = re.sub(r'[^\w\s/-]', '', text)
    return text

In [11]:
patterns = [
    r'^\d{1,3}/\d{1,3}/\d{1,3}$',  # Pattern for 1-3 digits/numbers/1-3 digits/numbers/1-3 digits/numbers
    r'^\d{1,3}-\d{1,3}-\d{1,3}$',  # Pattern for 1-3 digits-numbers-1-3 digits-numbers-1-3 digits-numbers
    r'.*gangbild.*',               # Pattern for strings containing "gangbild" (case-insensitive)
    r'.*druck.*',                  # Pattern for strings containing "druck" (case-insensitive)
    r'.*ergu.*',                   # Pattern for strings containing "ergu" (case-insensitive)
    r'.*schwell.*',                # Pattern for strings containing "schwell" (case-insensitive)
    r'.*rötung.*',                # Pattern for strings containing "schwell" (case-insensitive)
    r'.*überstreckung.*',                # Pattern for strings containing "schwell" (case-insensitive)
]

# Combine patterns into a single regular expression
combined_pattern = r'|'.join(patterns)

In [14]:


# Filter the list
for idx, row in df.iterrows():
    parts = custom_split(row['Text_Bef'], do_not_split)  # Custom split by exceptions
    parts = list(map(preprocess, parts))
    df.at[idx, 'All'] = ', '.join(parts)
    
    filtered_parts = [s for s in parts if s and re.match(combined_pattern, s, re.IGNORECASE)]
    
    df.at[idx, 'Formatted'] = ', '.join(filtered_parts)

In [15]:
df.head(10)

,Patientid_x,Text_Bef,Text_ANA,Geschlecht,Diagnose,All,Formatted
0,89078.0,"li Knie: diskr. Erguss, kein DS med. GS, kein ...","Schmerzen werden besser, geht heute das dritte...",männlich,Meniskus,"links knie diskr erguss, kein druckschmerz med...","links knie diskr erguss, kein druckschmerz med..."
1,76793.0,"Re. Knie: deutlicher Erguss, 0-5-110, reizlose...",Am 02.03. beim Trampolinspringen Distorsionstr...,männlich,Meniskus,"rechts knie deutlicher erguss, keine überstrec...","rechts knie deutlicher erguss, keine überstrec..."
2,409850.0,"Linkes Kniegelenk: Extension/Flexion 0/3/100°,...","wv mit aktuellem MRT. Beugung besser, Streckun...",männlich,Meniskus,linkes kniegelenk extension/flexion keine über...,linkes kniegelenk extension/flexion keine über...
3,76793.0,"linke Schulter: GhAbd. 90°, Aro 60°, Iro LWK 1...",Patient berichtet er habe zwischenzeitlich ern...,männlich,Meniskus,"linke schulter ghabd, 90, aro 60, iro lwk 1, j...",
4,91686.0,"re.Knie: ROM 0-0-140, Lachman neg. fest, DS me...","Belastungserprobung mit Rennrad und Laufen, bi...",männlich,Meniskus,reknie rom keine überstreckung des knies kein ...,reknie rom keine überstreckung des knies kein ...
5,113302.0,"Linkes Kniegelenk: Extension/Flexion 0/2/135°,...",Heute Wv bei uns nach 3xAC Spülung nach Gelenk...,männlich,Meniskus,linkes kniegelenk extension/flexion keine über...,linkes kniegelenk extension/flexion keine über...
6,305785.0,Beinachse gerade Linkes Kniegelenk: Extension/...,Patient hat starke Schmerzen im linken Knie wä...,männlich,Meniskus,beinachse gerade linkes kniegelenk extension/f...,beinachse gerade linkes kniegelenk extension/f...
7,91686.0,Beinachse gerade Rechtes Kniegelenk: Extension...,Seit dem 09.06. nach Kniedistorsion Schmerzen ...,männlich,Meniskus,beinachse gerade rechtes kniegelenk extension/...,beinachse gerade rechtes kniegelenk extension/...
8,63950.0,"li.Knie:keine Rötung, keine Überwärmung, deutl...","WV ca. 2,5 Monate postoperativ.",männlich,Meniskus,"likniekeine rötung, keine überwärmung, deutlic...","likniekeine rötung, deutlicher gelenkerguss, k..."
9,113302.0,"Beinachse gerade, Beinlänge gleich bzs. re. fr...",1. tag Post-Op nach AM-Refixation bei luxierte...,männlich,Meniskus,"beinachse gerade, beinlänge gleich bzs, rechts...",5 cm länger linkes kniegelenk extension/flexio...


In [16]:
# Define the patterns
df.to_excel("./data/formatted_examinations.xlsx", index=False)

In [13]:
val = "Beinachse gerade Rechtes Kniegelenk: Extension/Flexion 0/25/110° (federnde Extensionshemmung), kein Gelenkerguss, Lachman neg. fest, Seitenbänder stabil, Druckschmerz seitlicher Gelenkraum. unauffällige Patella."
parts = custom_split(val, do_not_split)  # Custom split by exceptions
parts = list(map(preprocess, parts))
print(parts)
    
    

['beinachse gerade rechtes kniegelenk extension/flexion keine überstreckung des knies streckdefizit des knies angemessene beugung des knies federnde extensionshemmung', 'kein gelenkerguss', 'lachman neg fest', 'seitenbänder stabil', 'druckschmerz seitlicher gelenkraum', 'unauffällige patella']


In [14]:
filtered_parts = [s for s in parts if s and re.match(combined_pattern, s, re.IGNORECASE)]
print(filtered_parts)

['beinachse gerade rechtes kniegelenk extension/flexion keine überstreckung des knies streckdefizit des knies angemessene beugung des knies federnde extensionshemmung', 'kein gelenkerguss', 'druckschmerz seitlicher gelenkraum']
